In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

filepath_processed = 'C:/Users/Dirk/Google Drive/Dirk/TU/Afstuderen/Afstuderen/Thesis/3. Formalisation/Processed CBS data/'
filepath_raw = 'C:/Users/Dirk/Google Drive/Dirk/TU/Afstuderen/Afstuderen/Thesis/3. Formalisation/Raw CBS data/'
filepath_other = 'C:/Users/Dirk/Google Drive/Dirk/TU/Afstuderen/Afstuderen/Thesis/3. Formalisation/Raw data other/'

In [2]:
#timeit.timeit(pd.read_csv(filepath_processed + 'Total_dataset.csv'))
start = datetime.now()
data =  pd.read_csv(filepath_processed + 'Total_dataset.csv', index_col = 0, 
                    dtype = {'gemeente': str ,'postcode': str ,'huisnummer' : int ,
                             'huisletter': str ,'huisnummertoevoeging': str , 
                             'oppervlakteverblijfsobject' : float ,'pandbouwjaar' : int ,
                             'openbareruimte' : str ,'pandstatus' : str , 'Woningkenmerken': str ,
                             'Energielabel' : str,'leeftijd' : int ,'Age group': str, 
                             'GebruiksOppervlakteklasse' : str,'Bouwjaarklasse' : str,
                             'gasverbruik m3/m2' : float ,'geschat gasverbruik jaarlijks' : float, 
                             'Gemiddelde grootte huishouden' : float , 
                             'elektriciteits verbruik pp' : float , 
                             'geschat elektriciteitsverbruik jaarlijks' : float})

print ('Loading huge dataset took: ' + str((datetime.now()-start).total_seconds()) + ' seconds')

buurten = pd.read_csv(filepath_raw + 'postcode_huisnummer_buurten.csv',delimiter = ';')
wijk = pd.read_csv(filepath_raw + 'wijk2019.csv', delimiter = ';')
buurt = pd.read_csv(filepath_raw + 'buurt2019.csv', delimiter = ';')


C:\Users\Dirk\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\Dirk\Anaconda3\lib\site-packages\numpy\lib\arraysetops.py:472: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


Loading huge dataset took: 33.644887 seconds


In [3]:
wijk_dict = wijk.astype(str).set_index('Wijkcode2019').to_dict()['Wijknaam_2019K_NAAM']
buurt.Buurtcode_2019 = buurt.Buurtcode_2019.astype(str)
buurt_dict = buurt.set_index('Buurtcode_2019').to_dict()['Buurtnaam_2019']

buurten[['Buurt2019']]= buurten[['Buurt2019']].astype(str)
buurten[['Wijk2019']] = buurten[['Wijk2019']].astype(str)
buurten['buurt_naam'] = buurten['Buurt2019'].map(buurt_dict)
buurten['wijk_naam'] = buurten['Wijk2019'].map(wijk_dict)
buurten = buurten.drop(['Buurt2019','Wijk2019','Gemeente2019'], axis = 1).rename(columns = {'PC6':'postcode','Huisnummer':'huisnummer'})

In [4]:
#Hier aanpassing
data = data.merge(buurten, on = ['postcode','huisnummer'], how = 'inner')

In [6]:
data2 = data.drop(['pandstatus','Bouwjaarklasse', 'Gemiddelde grootte huishouden', 'elektriciteits verbruik pp', 'leeftijd','openbareruimte','postcode', 'huisnummer', 'huisnummertoevoeging','huisletter','pandbouwjaar'],axis =1)

In [7]:
investment_cost = pd.read_csv(filepath_other + 'investment cost CE_Delft.csv')
energy_savings = pd.read_csv(filepath_other + 'Energy savings CE_Delft.csv')
investment_cost.Schil = investment_cost.Schil.str.strip('Huidig ')
energy_savings.Schil = energy_savings.Schil.str.strip('Huidig ')
i_dict = investment_cost.set_index(['Schil', 'Woningtype']).to_dict()
e_dict = energy_savings.set_index('Schil').to_dict()

In [8]:
energy_savings.Schil[0:7]

0    G
1    F
2    E
3    D
4    C
5    B
6    A
Name: Schil, dtype: object

In [9]:
stapel_dict = {'Appartement':'Gestapeld','2-onder-1-kapwoning':'Grondgebonden','Hoekwoning':'Grondgebonden','Tussenwoning':'Grondgebonden','Vrijstaande woning':'Grondgebonden','O':'Grondgebonden','N':'Grondgebonden','C0':'Grondgebonden'}
data2['Type'] = data2['Woningkenmerken'].map(stapel_dict)
gamma_vars = []
for i in energy_savings.Schil[1:7]:
    for j in ['Grondgebonden','Gestapeld']:

        data2['Yearly savings ' + i + ' (m3)'] = data2['geschat gasverbruik jaarlijks'] * data2['Energielabel'].map(e_dict[i])
        data2['Total cost '+ i ] = data2['oppervlakteverblijfsobject'] * pd.Series(list(zip(data2['Energielabel'],data2['Type']))).map(i_dict[i])
        data2['Savings / cost ' + i] = data2['Yearly savings ' + i + ' (m3)'] / data2['Total cost '+ i]
        gamma_vars.extend(['Total cost '+ i,'Savings / cost ' + i])

In [10]:
#For calculating gamma distribution
def alpha(x):
    if x.var() > 0:
        return x.mean()**2 / x.var()
    else:
        return 1
def theta(x):
    if x.var() > 0:
        return x.var() / x.mean()
    else:
        return 1
    
agg_dict = {}
for i in gamma_vars :
    agg_dict[i] = ['mean', alpha, theta]

#data3.groupby(by = ['gemeente','buurt_naam']).agg(['min','mean','max','std', alpha, theta])
data3 = data2.drop(['oppervlakteverblijfsobject','Energielabel','geschat elektriciteitsverbruik jaarlijks','geschat gasverbruik jaarlijks', 'gasverbruik m3/m2'],axis =1)

gamma_dist = data3.groupby(by = ['gemeente','buurt_naam']).agg(agg_dict)#{gamma_vars : ['mean', alpha, theta]})

In [11]:
gamma_dist.to_csv(filepath_processed + 'total_gamma.csv')

In [12]:
age_group = data3.groupby(by = ['gemeente','buurt_naam'])['Age group'].value_counts()#,'Age group'], sort = False).count()['Woningkenmerken'].reset_index()
woningkenmerken = data3.groupby(by = ['gemeente','buurt_naam'])['Woningkenmerken'].value_counts()
oppervlakte = data3.groupby(by = ['gemeente','buurt_naam'])['GebruiksOppervlakteklasse'].value_counts()
age_group = age_group.to_frame().unstack().fillna(0)['Age group'][['Age0to9','Age10to19','Age20to29','Age30to39','Age40to49','Age50to59','Age60to69','Age70to79','Age80to89','Age90to99','Age100plus']]#.reset_index()
woningkenmerken = woningkenmerken.to_frame().unstack().fillna(0)['Woningkenmerken'][['2-onder-1-kapwoning', 'Appartement', 'Hoekwoning', 'Tussenwoning', 'Vrijstaande woning']]
oppervlakte = oppervlakte.to_frame().unstack().fillna(0)['GebruiksOppervlakteklasse'][['15 tot 50 m2','50 tot 75 m2', '75 tot 100 m2', '100 tot 150 m2', '150 tot 250 m2', '250 tot 500 m2']]
total = data3.groupby(by = ['gemeente','buurt_naam']).count().max(axis = 1)

In [13]:
for i in woningkenmerken.columns:
    woningkenmerken[i] = woningkenmerken[i] / total
for i in age_group.columns:
    age_group[i] = age_group[i] / total
for i in oppervlakte.columns:
    oppervlakte[i] = oppervlakte[i] / total

In [14]:
pd.concat([total, woningkenmerken,oppervlakte,age_group], axis = 1).rename(columns = {0 :'total'}).to_csv(filepath_processed + 'total_characteristics.csv')